In [6]:
import selenium
from selenium import webdriver
import pandas as pd
from selenium.webdriver import Chrome
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import time
import uuid
import os
import urllib
import json


class Scraper:
    def __init__(self, url: str = 'https://www.propertypal.com') -> None:
        self.driver = Chrome(ChromeDriverManager().install())
        self.driver.get(url)
        
    def ivan_accept_cookies(self, xpath:str = '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]'):
        
        try:
            time.sleep(1)
            WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))

            button = self.driver.find_element(By.XPATH, xpath)
            button.click()
        except TimeoutException:
            print("No Cookies Found")

    def accept_cookies(self, xpath:str = '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]'):
        button = self.driver.find_element(By.XPATH, xpath)
        button.click()

    
    def search_word(self, xpath, Postcode = "BT3"):

        search = self.driver.find_element(By.XPATH, xpath)
        self.driver.implicitly_wait(10)
        ActionChains(self.driver).move_to_element(search).click(search).perform()
        ActionChains(self.driver).send_keys(Postcode).perform()

    def scroll_down(self):
        
        self.driver.execute_script("window.scrollTo(0, 540)") 
    
    def search_rent(self):
        rent = self.driver.find_element_by_xpath('//*[@id="searchForm"]/div/div[2]/button[2]')
        rent.click()

    def search_sale(self):
        rent = self.driver.find_element_by_xpath('//*[@id="searchForm"]/div/div[2]/button[1]')
        rent.click()

    def button_click(self,xpath):
        button = self.driver.find_element_by_xpath(xpath)
        button.click()
    
    def find_container(self, xpath: str = '//*[@id="body"]/div[3]/div/div[1]/div/ul'):
        self.container = self.driver.find_element(By.XPATH, xpath)
        return self.container

##
# NAVIGATE THE WEBSITE
##

if __name__ == "__main__":
    bot = Scraper()
    bot.ivan_accept_cookies()
    time.sleep(2)
    bot.button_click('//a[@href="/login"]')
    time.sleep(2)
    bot.search_word('//input[@placeholder="Email address"]','sopranotony233@gmail.com')
    time.sleep(2)
    bot.search_word('//input[@placeholder="Password"]','sopranotony321')
    time.sleep(2)
    bot.button_click('//*[@id="root"]/div/div/div/div/div/div[1]/div/div[2]/div[2]/form/button')
    time.sleep(2)
    bot.button_click('//a[@class="mainnav-logo"]')  
    time.sleep(2)
    bot.search_word('//*[@id="searchForm"]/div/div[1]')
    time.sleep(2)
    bot.search_rent()
    time.sleep(2)
    

    #CREATE THE LIST OF LINKS
    list_links = []
    print("Finding elements...")
    while True:
        container = bot.find_container()
        items = container.find_elements(By.XPATH, './li')
        for i in items:
            try:
                house = i.find_element(By.XPATH, './/a[2]')
                link = house.get_attribute('href')
                list_links.append(link)
            except:
                print("no href found")
        
        try:
            bot.button_click('//a[@class="btn paging-next"]')
        except NoSuchElementException:
            print("end of list")
            break

    #GRAB INFO FROM EACH LINK AND STORE

    prop_dict = {"fr-id": [],
            "id": [],
            "Link": [],
             "Summary": [],
            "Address": [],
            "Price": [],
            "Image links": []
            }
    for link in list_links:
        im = link[-6:]
        im2 = link[28:35]
        prop_dict["fr-id"].append(im2+im)
        id = uuid.uuid4()
        prop_dict["id"].append(id)
        bot.driver.get(link)
        prop_dict["Link"].append(link)
        time.sleep(1)
        summary = bot.driver.find_element(By.XPATH, '//div[@class="prop-heading-brief"]')
        prop_dict["Summary"].append(summary.text)
        time.sleep(1)
        info = bot.driver.find_element(By.XPATH, '//div[@class="prop-summary-row"]')
        address = info.find_element(By.XPATH, './/h1')
        prop_dict["Address"].append(address.text)
        time.sleep(1)
        price = bot.driver.find_element(By.XPATH, '//div[@class="prop-price"]')
        prop_dict["Price"].append(price.text)
        container_2= bot.driver.find_element(By.XPATH, '//div[@class="Slideshow-slides SlideshowCarousel"]')
        items_2 = container_2.find_elements(By.XPATH, './/img')
        img_links = []
        for i in items_2:
            try:
                link = i.get_attribute('src')
                img_links.append(link)
            except:
                print('No src found')
        prop_dict["Image links"].append(img_links)


    Postcode = "BT3"
    os.mkdir(f"/Users/ryanhughes/Desktop/Aicore/Property-Pal-Pipeline-/Property_Photos/{Postcode}")
    image_directory = os.path.dirname(f"/Users/ryanhughes/Desktop/Aicore/Property-Pal-Pipeline-/Property_Photos/{Postcode}/")
    img_link_ct=0
    for i in prop_dict["Image links"]:
        img_link_ct += 1
        img_ct = 0
        for url in i:
            try:
                img_ct += 1
                req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
                with open(f"{image_directory}/{prop_dict['fr-id'][img_link_ct]}_{str(img_ct)}.jpg", "wb") as f:
                    with urllib.request.urlopen(req) as r:
                        f.write(r.read())
            except IndexError:
                pass

df = pd.DataFrame(prop_dict)
df

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/ryanhughes/.wdm/drivers/chromedriver/mac64/103.0.5060.53/chromedriver] found in cache
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_1068/231070320.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = Chrome(ChromeDriverManager().install())
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_1068/231070320.py:60: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = self.driver.find_element_by_xpath(xpath)
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_1068/231070320.py:52: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  rent = self.driver.find_element_by_xpath('//*[@id="searchF

Finding elements...
no href found
no href found
end of list


,fr-id,id,Link,Summary,Address,Price,Image links
0,4-18-th767245,ddf737ad-9d9f-4cd9-98ee-2725aea5514e,https://www.propertypal.com/4-18-the-arc-titan...,2 Bed Apartment To Rent,"4-18 THE ARC, TITANIC QUARTER, 2C QUEENS ROAD,...","£1,250\n/ month + fees",[https://media.propertypal.com/sd/p/766045/323...
1,apt-34-765126,f1372420-a95e-4774-9328-211ffdd4dd5a,https://www.propertypal.com/apt-34-quay-gate-s...,3 Bed Apartment To Rent,"APT 34, QUAY GATE,\nStation Street, Belfast BT...",£925\n/ month + fees,[https://media.propertypal.com/sd/p/763926/322...
2,29-quay662727,45e0c3e0-0bf1-47d3-a234-e2f76ab99dfb,https://www.propertypal.com/29-quaygate-statio...,1 Bed Apartment To Rent,"29 QUAYGATE,\nStation Street , Belfast BT3 9DA",£775\n/ month + fees,[https://media.propertypal.com/sd/p/661527/322...
3,81-quay763199,d7936435-2414-41e3-b231-c6b3588fdfee,https://www.propertypal.com/81-quay-gate-19-st...,2 Bed Apartment To Rent,"81 QUAY GATE, 19 STATION STREET,\nBelfast BT3 9DB",£875\n/ month + fees,[https://media.propertypal.com/sd/p/761999/320...
4,the-arc762119,0d1d08a0-5397-4d09-b2ce-fc432777b1ad,https://www.propertypal.com/the-arc-titanic-qu...,2 Bed Apartment To Rent,"THE ARC, TITANIC QUARTER, BELFAST,\nBelfast BT...","£1,600\n/ month + fees",[https://media.propertypal.com/sd/p/760919/319...
5,apartme758485,49294cb6-7aeb-478b-8a23-7dc9745d1aa3,https://www.propertypal.com/apartment-the-arc-...,1 Bed Apartment To Rent,"APARTMENT @ THE ARC (BLOCK 7), 2H QUEENS ROAD,...",£830\n/ month + fees,[https://media.propertypal.com/handheld/p/7572...


In [5]:
df = pd.DataFrame(prop_dict)
df

ValueError: All arrays must be of the same length

In [1]:
from json import JSONEncoder
from uuid import UUID

old_default = JSONEncoder.default

def new_default(self, obj):
    if isinstance(obj, UUID):
        return str(obj)
    return old_default(self, obj)

JSONEncoder.default = new_default

In [7]:
import json  
with open('/Users/ryanhughes/Desktop/Aicore/Property-Pal-Pipeline-/raw_data/data.json', 'w') as f:
    json.dump(prop_dict, f)

In [9]:
os.mkdir(f"/Users/ryanhughes/Desktop/Aicore/Property-Pal-Pipeline-/Test")

In [138]:
import os
Postcode = "BT4"
os.mkdir(f"/Users/ryanhughes/Desktop/Aicore/Property-Pal-Pipeline-/Property_Photos/{Postcode}")
image_directory = os.path.dirname(f"/Users/ryanhughes/Desktop/Aicore/Property-Pal-Pipeline-/Property_Photos/{Postcode}/")
img_link_ct=0
for i in prop_dict["Image links"]:
    img_link_ct += 1
    img_ct = 0
    for url in i:
        
        img_ct += 1
        req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        with open(f"{image_directory}/{prop_dict['fr-id'][img_link_ct]}_{str(img_ct)}.jpg", "wb") as f:
            with urllib.request.urlopen(req) as r:
                f.write(r.read())


IndexError: list index out of range

In [141]:
prop_dict

{'fr-id': ['56-park766769',
  'apt-1-2548636',
  '1a-care765807',
  '13-bath765788',
  'flat-3-532728',
  '229-par765499',
  '9-pims-765427',
  'apt-3-g765358',
  '53-uppe765329',
  '4-devon765273',
  '40-norw765201',
  'the-lim105704',
  '79-uppe730603',
  'apt-12-762260',
  '73-uppe761430',
  '352-bel512687',
  '84-lark759906',
  '9-larkf759461',
  '85-uppe606480',
  '55c-vic752602',
  '187-hol343908'],
 'id': [UUID('6bfb1391-bcbb-4bd2-9556-b48fee4f7ba8'),
  UUID('80563e07-e815-4366-ad17-5c3ec5f19629'),
  UUID('5eb317b2-083d-4fcc-9470-6f12b02213c0'),
  UUID('20f0b8d9-5112-4cfb-a6cd-39c6a302e579'),
  UUID('14cfb161-d95d-4b15-bf0b-5176a3a647b7'),
  UUID('b1cfebfa-295e-47fa-8139-900100e76b1f'),
  UUID('f8aa39e6-0cd2-40e1-ae0b-4ba984450cc9'),
  UUID('9e76cea7-115f-4355-b560-98aa282b36d2'),
  UUID('bafd28e8-3445-4f65-8bdc-729a1ff20849'),
  UUID('23b9af63-8a5c-4091-8d9c-3228f0c4f9a2'),
  UUID('3e9f32cb-56f6-462d-a484-d100843f78ae'),
  UUID('7c060c7d-62ad-4693-8f7f-1e4846cbf89e'),
  UUID('4

In [109]:
os.mkdir('/Users/ryanhughes/Desktop/Aicore/Property-Pal-Pipeline-/Property_Photos')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ryanhughes/Desktop/Aicore/Property-PalPipeline-/Property_Photos'

In [102]:
import os
os.mkdir('/Users/ryanhughes/Desktop/Aicore/Property-PalPipeline-/Property_Photos')
img_link_ct=0
for i in prop_dict["Image links"]:
    img_link_ct += 1
    img_ct = 0
    for url in i:
        
        img_ct += 1
        req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        with open(prop_dict["fr-id"][img_link_ct]+"_"+str(img_ct)+".jpg", "wb") as f:
            with urllib.request.urlopen(req) as r:
                os.path.join(r.read())

In [ ]:
prop_dict["image links"]

In [54]:
im = list_links[0][-6:]

im2 = list_links[0][28:35]

print(im+im2)

print(list_links[0])

76676956-park
https://www.propertypal.com/56-park-avenue-belfast/766769


In [2]:
import selenium
from selenium import webdriver
import pandas as pd
from selenium.webdriver import Chrome
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import time


class Scraper:
    def __init__(self, url: str = 'https://www.propertypal.com') -> None:
        self.driver = Chrome(ChromeDriverManager().install())
        self.driver.get(url)
        
    def ivan_accept_cookies(self, xpath:str = '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]'):
        
        try:
            time.sleep(1)
            WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))

            button = self.driver.find_element(By.XPATH, xpath)
            button.click()
        except TimeoutException:
            print("No Cookies Found")

    def accept_cookies(self, xpath:str = '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]'):
        button = self.driver.find_element(By.XPATH, xpath)
        button.click()

    
    def search_word(self, xpath, text):

        search = self.driver.find_element(By.XPATH, xpath)
        self.driver.implicitly_wait(10)
        ActionChains(self.driver).move_to_element(search).click(search).perform()
        ActionChains(self.driver).send_keys(text).perform()

    def scroll_down(self):
        
        self.driver.execute_script("window.scrollTo(0, 540)") 
    
    def search_rent(self):
        rent = self.driver.find_element_by_xpath('//*[@id="searchForm"]/div/div[2]/button[2]')
        rent.click()

    def search_sale(self):
        rent = self.driver.find_element_by_xpath('//*[@id="searchForm"]/div/div[2]/button[1]')
        rent.click()

    def button_click(self,xpath):
        button = self.driver.find_element_by_xpath(xpath)
        button.click()
    
    def find_container(self, xpath: str = '//*[@id="body"]/div[3]/div/div[1]/div/ul'):
        self.container = self.driver.find_element(By.XPATH, xpath)
        return self.container

In [61]:
bot = Scraper()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/ryanhughes/.wdm/drivers/chromedriver/mac64/103.0.5060.53]
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_49062/100464251.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = Chrome(ChromeDriverManager().install())


In [76]:
container_2= bot.driver.find_element(By.XPATH, '//div[@class="Slideshow-slides SlideshowCarousel"]')
items = container_2.find_elements(By.XPATH, './/img')

[<selenium.webdriver.remote.webelement.WebElement (session="311458f89aee918ba583c6341a5e6c50", element="a9a330fe-0d75-4536-90a3-806426284c6f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="311458f89aee918ba583c6341a5e6c50", element="f025e020-77bb-4171-8f52-a740d75a4429")>,
 <selenium.webdriver.remote.webelement.WebElement (session="311458f89aee918ba583c6341a5e6c50", element="cc7adcbf-1f3e-447c-b157-01622ab65921")>,
 <selenium.webdriver.remote.webelement.WebElement (session="311458f89aee918ba583c6341a5e6c50", element="4aa129fd-1ff7-4310-b277-e5a45a01c172")>]

In [66]:
items = bot.find_container('//div[@class="Slideshow-slides SlideshowCarousel"]')


In [77]:
container_2= bot.driver.find_element(By.XPATH, '//div[@class="Slideshow-slides SlideshowCarousel"]')
items = container_2.find_elements(By.XPATH, './/img')
img_links = []
for i in items:
    try:
        link = i.get_attribute('src')
        img_links.append(link)
    except:
        print('No src found')
    

In [152]:
bot.driver.get(img_links[0])

WebDriverException: Message: chrome not reachable
  (Session info: chrome=103.0.5060.114)
Stacktrace:
0   chromedriver                        0x000000010bfea079 chromedriver + 4444281
1   chromedriver                        0x000000010bf76403 chromedriver + 3970051
2   chromedriver                        0x000000010bc10eef chromedriver + 409327
3   chromedriver                        0x000000010bc00112 chromedriver + 340242
4   chromedriver                        0x000000010bc00afe chromedriver + 342782
5   chromedriver                        0x000000010bc029b2 chromedriver + 350642
6   chromedriver                        0x000000010bbfb32c chromedriver + 320300
7   chromedriver                        0x000000010bc12452 chromedriver + 414802
8   chromedriver                        0x000000010bc768db chromedriver + 825563
9   chromedriver                        0x000000010bc64683 chromedriver + 751235
10  chromedriver                        0x000000010bc3aa45 chromedriver + 580165
11  c

In [151]:
img_links

['https://media.propertypal.com/sd/p/765569/32335126.jpg',
 'https://media.propertypal.com/sd/p/765569/32335125.jpg',
 'https://media.propertypal.com/sd/p/765569/32335129.jpg',
 'https://media.propertypal.com/sd/p/765569/32335135.jpg']

In [57]:
import urllib.request

url = img_links[0]

r = urllib.request.urlopen(url)
with open("test_img.jpg", "wb") as f:
    f.write(r.read())

HTTPError: HTTP Error 403: Forbidden

In [59]:
import urllib.request
for url in img_links:

    req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    with open("image", "wb") as f:
        with urllib.request.urlopen(req) as r:
            f.write(r.read())

In [60]:

img_links

['https://media.propertypal.com/sd/p/764607/32259497.jpg',
 'https://media.propertypal.com/sd/p/764607/32259499.jpg',
 'https://media.propertypal.com/sd/p/764607/32259501.jpg',
 'https://media.propertypal.com/sd/p/764607/32259505.jpg']

In [43]:
list_UUID = []
for i in list_links:
    id = uuid.uuid4()
    list_UUID.append(id)
list_UUID

[UUID('2d80ad2f-786c-483d-92aa-fd1677316891'),
 UUID('0e02d07c-d303-4884-bd57-034708a8c6c4'),
 UUID('78a47e88-29f0-49af-bf41-1d227b4664e7'),
 UUID('fece02a6-f83a-49b8-b405-76a2da47dd64'),
 UUID('4ceb9dfd-6ca4-490a-bded-d8ea198fe463'),
 UUID('a296d491-8e7f-4f8a-9620-5df711cab5f3'),
 UUID('adfb8360-b539-437c-8daf-36bd135ffb6b'),
 UUID('0db7e6b0-c827-419c-9692-8b1e2d133dec'),
 UUID('7709d2bd-cf4f-4bf0-b410-7ce7e5d168fe'),
 UUID('f48a90b9-54f1-41da-a634-9ecaace25380'),
 UUID('398e2572-a765-4862-8bf2-0f1604a1116c'),
 UUID('4a838e38-f1e1-43a8-814e-bb588af1b32a'),
 UUID('4cd3f1ac-b3af-43f9-a119-de591268a2d9'),
 UUID('c3d31853-bcd7-44a7-8510-5239fe754420'),
 UUID('f52ae5b0-725a-48cb-b7e5-d013b38a54d3'),
 UUID('d9434206-a50c-48bb-95b9-1c1c116ff394'),
 UUID('ee488d6d-7e6e-47e6-be7c-72ba702b09d7'),
 UUID('d4e1eec9-7c53-49f3-8e63-48164c767b05'),
 UUID('5fa99b9d-a07c-4e3e-a6a5-0da11dae2a32'),
 UUID('6ad2dce6-a406-456a-9738-4238c070df51'),
 UUID('eafe051f-cc03-4810-88b5-ae215408cd38'),
 UUID('28b7b1